In [83]:
import pandas as pd
import glob
import os

# 1. Đặt đường dẫn tới thư mục chứa các file Excel
folder_path = ''  # Thay đổi thành đường dẫn thực tế

# 2. Danh sách tên file cần lấy (bao gồm cả phần mở rộng)
file_names_to_read = ['noun.feeling.xlsx', 'verb.emotion.xlsx']  # Thay đổi thành danh sách file bạn cần

# 3. Tạo danh sách đầy đủ đường dẫn tới file
file_paths_to_read = [os.path.join(folder_path, name) for name in file_names_to_read]

# 4. Tạo một danh sách để lưu các DataFrame
df_list = []

# 5. Duyệt qua từng file trong danh sách file cụ thể và đọc dữ liệu
for file in file_paths_to_read:
    if os.path.exists(file):  # Kiểm tra xem file có tồn tại không
        try:
            # Đọc file Excel, giả sử bạn muốn đọc sheet đầu tiên
            df = pd.read_excel(file, header=None)

            # Xóa các cột thừa (cột toàn giá trị NaN do dấu phẩy thừa)
            df = df.dropna(axis=1, how='all')

            # Thêm một cột mới để biết nguồn file (tùy chọn)
            df['Nguồn_File'] = os.path.basename(file)

            # Thêm DataFrame vào danh sách
            df_list.append(df)

            print(f"Đã đọc thành công: {file}")

        except Exception as e:
            print(f"Lỗi khi đọc file {file}: {e}")
    else:
        print(f"File không tồn tại: {file}")

# 6. Nối tất cả các DataFrame lại với nhau theo hàng
if df_list:  # Kiểm tra nếu có dữ liệu trong danh sách
    combined_df = pd.concat(df_list, ignore_index=True)
    print("Dữ liệu đã được kết hợp thành công.")
else:
    print("Không có dữ liệu để kết hợp.")
#
# combined_df.to_excel('wordnet_emotion_felling_adj.xlsx', index=False)


combined_df = pd.read_excel('wordnet_full.xlsx')
# 6. Xóa cột 'Nguồn_File' nếu không cần thiết
if 'Nguồn_File' in combined_df.columns:
    combined_df = combined_df.drop(columns=['Nguồn_File'])

# 7. Xây dựng từ điển đồng nghĩa
synonym_dict = {}

for index, row in combined_df.iterrows():
    # Lấy danh sách từ trong hàng, loại bỏ NaN và loại bỏ khoảng trắng
    words = [word.strip().lower().replace(" ", "_") for word in row.dropna().values if isinstance(word, str)]
    # Bỏ qua các hàng có ít hơn 2 từ
    # if len(words) < 2:
    #     continue
    # Thêm mỗi từ vào từ điển, ánh xạ đến các từ còn lại trong hàng
    for word in words:
        if word in synonym_dict:
            # Nếu từ đã có trong từ điển, thêm các từ mới vào danh sách đồng nghĩa
            synonym_dict[word].update(words)
        else:
            # Tạo một tập hợp mới cho từ này
            synonym_dict[word] = set(words)

# Loại bỏ từ gốc khỏi danh sách đồng nghĩa của nó
for word in synonym_dict:
    synonym_dict[word].discard(word)


# 8. Hàm truy xuất từ đồng nghĩa
def get_synonyms(query_word):
    query_word = query_word.strip().lower()
    if query_word in synonym_dict:
        synonyms = synonym_dict[query_word]
        return list(synonyms)
    else:
        return None

Đã đọc thành công: noun.feeling.xlsx
Đã đọc thành công: verb.emotion.xlsx
Dữ liệu đã được kết hợp thành công.


In [84]:
import numpy as np

filename = 'vietnamese-stopwords.csv'
data = pd.read_csv(filename, names=['word'])
list_stopwords = data['word']
print(list_stopwords)
myarray = np.asarray(list_stopwords)
print(myarray)

0         a lô
1         a ha
2           ai
3        ai ai
4       ai nấy
         ...  
1797     ừ nhé
1798     ừ thì
1799      ừ ào
1800       ừ ừ
1801         ử
Name: word, Length: 1802, dtype: object
['a lô' 'a ha' 'ai' ... 'ừ ào' 'ừ ừ' 'ử']


In [85]:
import string
import regex as re
import numpy as np
import pandas as pd
from underthesea import word_tokenize, text_normalize

# =======================
# Vietnamese Character Normalization Functions
# =======================

# Define Unicode and Unsigned Characters
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    """
    Load a dictionary mapping accented Vietnamese characters to their base forms.
    """
    dic = {}
    # Define the list of accented characters (1252 encoding)
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|' \
              'ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|' \
              'ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|' \
              'À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|' \
              'È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|' \
              'Ì|Í|Ỉ|Ĩ|Ị|' \
              'Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|' \
              'Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|' \
              'Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')

    # Define the list of UTF-8 encoded characters
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|" \
              "ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|" \
              "ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|" \
              "À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|" \
              "È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|" \
              "Ì|Í|Ỉ|Ĩ|Ị|" \
              "Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|" \
              "Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|" \
              "Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')

    # Create the dictionary mapping
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

dicchar = loaddicchar()

def covert_unicode(txt):
    """
    Convert accented Vietnamese characters to their base forms.
    """
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|' \
        r'è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|' \
        r'ì|í|ỉ|ĩ|ị|' \
        r'ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|' \
        r'ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|' \
        r'ỳ|ý|ỷ|ỹ|ỵ|' \
        r'À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|' \
        r'È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|' \
        r'Ì|Í|Ỉ|Ĩ|Ị|' \
        r'Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|' \
        r'Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|' \
        r'Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# =======================
# Existing Vietnamese Accent Normalization Functions
# =======================

# Define mappings for Vietnamese vowels and their accents
# Note: Ensure that bang_nguyen_am and nguyen_am_to_ids are fully defined
bang_nguyen_am = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ'],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ'],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị'],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ'],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ'],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ'],
    # Add more vowels if necessary
]

# Create a mapping from vowel to its indices in bang_nguyen_am
nguyen_am_to_ids = {}
for idx, vowel_group in enumerate(bang_nguyen_am):
    for jdx, vowel in enumerate(vowel_group):
        nguyen_am_to_ids[vowel] = (idx, jdx)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
            if not qu_or_gi or index != 1:
                nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1], (-1, -1))
                if x != -1:
                    chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids.get(chars[index], (-1, -1))
        if x == 4 or x == 8:  # Example indices for specific vowels like 'ê', 'ơ'
            chars[index] = bang_nguyen_am[x][dau_cau]
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[0]], (-1, -1))
            if x != -1:
                chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[1]], (-1, -1))
            if x != -1:
                chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[1]], (-1, -1))
        if x != -1:
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
    Normalize Vietnamese sentence to the old typing standard.
    :param sentence: str, input Vietnamese sentence
    :return: str, normalized sentence
    """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        # Using regex to handle punctuation around words
        cw = re.sub(r'(^\p{P}*)([\p{L}]+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tieng_viet(cw[1])
            words[index] = ''.join(cw)
    return ' '.join(words)

# =======================
# Additional Unicode Normalization Function
# =======================

# The covert_unicode function replaces accented characters with their base forms
# This can be used before or after the existing normalization depending on your needs

# =======================
# Existing Text Preprocessing Functions
# =======================

# Load stopwords from CSV
filename = 'vietnamese-stopwords.csv'
data = pd.read_csv(filename, names=['word'])
list_stopwords = data['word'].tolist()  # Convert to list for easier checking
myarray = np.asarray(list_stopwords)

# Load additional stopwords from text file
stopwords_small = []
with open('stopwords-vi_news.txt', 'r', encoding='utf-8') as file:
    stopwords_small = file.readlines()

# Clean stopwords list
stopwords_small = [line.strip().replace(" ", "_") for line in stopwords_small]
list_rare_words_1 = [
        'fraction', 'altera', 'quad', 'cnpm', 'kaydotvn', 'daadotuitdotedudotvn', 'bohm', 'dotnet',
        'visual studio code', 'visual studio', 'visual code', 'visual', 'mutton quad', 'dreamweaver',
        'vertical fragmentation', 'i-ta-li-a', 'đbcl', 'crt', 'javabeans', 'if', 'for', 'while',
        'struct', 'pl', 'windows phone', 'pm', 'assembly', 'scrum_project', 'scrum project',
        'css', 'jquery', 'vector', 'gb', 'ram', 'ram', '...', 'codefun', 'egov', 'java', 'vmware',
        'placement', 'fork', 'round robin', 'patin', 'pattern', 'serverside', 'dotnet', 'c++',
        'javascript', 'ch', 'uit khoa', 'proteus', 'console', 'form', 'vật lý học', 'aep',
        'servlet', 'skype', 'doubledot', 'poster', 'everything', 'amp', 'app', 'seo', 'app',
        'progressive web', 'xim', 'win server', 'đối với', 'p', 's', 'p s', 'dropbox', 'bạn',
        'cplusplus', 'socket', 'sub', 'switch', 'tôi', 'em', 'các em', 'tụi em', 'chúng em', 'các bạn', 'tụi em', 'chúng em'
        'severside', 'network programing with csharp', 'dfd', 'naives bayes', 'naive', 'bayes', 'cs', 'js', 'max', 'elg', 'fix', 'proxy',
        'hub', 'bridge','switch', 'windows', 'turnitindotcom', 'extensive reading', 'reading', 'extensive', 'search', 'quick', 'file header','paper',
        'directx', 'windows', 'linux', 'vote', 'itdotf', 'router', 'silverlight', 'đa luồng', 'crack', 'wrede' ,'dbpedia','ontology', 'tmf', 'vhdl',
        'hdl', 'jsp', 'pđt', 'lisp', 'json', 'cpp', 'các_em', 'chúng_em', 'tụi_em', 'các_bạn', 'tụi_em', 'sinh_viên', 'là', 'và', 'thì', 'vì', 'mà', 'của', 'khi', 'như', 'lại', 'đó', 'đây', 'kia', 'ấy', 'sẽ', 'mình', 'nếu', 'vậy', 'rồi', 'với', 'bởi', 'mà', 'ấy', 'kia', 'sẽ', 'đó', 'dù', 'tuy', 'itp', 'forum', 'embeded', 'system', 'embeded system', 'embedded', 'titanium', 'blackbery', 'zun', 'phonegap', 'tizen', 'je', 'mediafire','toeic', 'ghz', 'cpu', 'module', 'datapath', 'papers', 'daa', 'dijktra', 'oracal', 'database', 'access', 'netbean', 'facebook', 'hackerrankdotcom', 'sort', 'multiagent', 'th', 'contemn', 'dbms', 'html', 'php', 'heapsort', 'khmtdotuitdotedudotvn', 'vdotv', 'engine', 'download','input', 'output', 'wtf','forum', 'poison', 'uit', 'career', 'như', 'version', 'outdoor', 'coursedotuitdotedudotvn', 'mini', 'matlab', 'standford', 'name', 'size', 'framework', 'ucla', 'comment', 'is','we','serverside', 'cassette', 'ios', 'android', 'scrum', 'itdote', 'xml', 'photo', 'down', 'unikey', '3dsmax', 'firmware', 'km','hackerrank', 'projectbase', 'er', 'gay', 'feed', 'mác – lênin', 'mác', 'lênin', 'coursedotuitdotedudotvn', 'nfc', 'chip', 'full', 'oi', 'ht', 'ubuntu', 'linux', 'it', 'wecode', 'code','oop', 'hướng đối tượng', 'cho','để','macbook', 'a_z', 'av', 'anh văn', 'đã','một','nhưng','học sinh'
    ]
# Clean stopwords list
stopwords_small = [line.strip().replace(" ", "_") for line in stopwords_small]
stopwords_small.append('dot')
stopwords_small.append('giảng_viên')
for item in list_rare_words_1:
    stopwords_small.append(item)

def preprocess_text_vietnamese_to_tokens(text):

    text = covert_unicode(text)

    text = chuan_hoa_dau_cau_tieng_viet(text)

    # 2. Chuyển đổi văn bản thành chữ thường
    text = text.lower()

    # 3. Thực hiện ánh xạ các cụm từ theo từ điển
    mapping_dict = {
        "thầy giáo": "giảng viên",
        "cô giáo": "giảng viên",
        "thầy": "giảng viên",
        "cô": "giảng viên",
        "giáo viên": "giảng viên",
        'vói': 'với',
        'giời': 'giờ',
        'nhiệt hình': 'nhiệt tình',
        'side': 'slide',
        'tân tình': 'tận tình',
        'teacher': 'giảng viên',
        'sadcolon': 'colonsad',
        'vi dụ': 'ví dụ',
        'easy': 'dễ',
        'so vời': 'so với',
        'tâp': 'tập',
        'av': 'tiếng anh',
        'nhannh': 'nhanh',
        'h': 'giờ',
        'đc': 'được',
        'dc': 'được',
        'smilesmile': 'smile',
        'tron': 'trong',
        'thướng': 'hướng',
        'nghung': 'nhưng',
        'chon': 'chọn',
        'them': 'thêm',
        'day': 'dạy',
        'midterm': 'giữa kỳ',
        'vi': 'vì',
        'quýêt': 'quyết',
        'teamwork': 'làm việc nhóm',
        'over time': 'quá giờ',
        'overtime': 'quá giờ',
        'ot': 'quá giờ',
        'outcome': 'mục tiêu',
        'basic': 'cơ bản',
        'check': 'kiểm tra',
        'gmail': 'email',
        'mail': 'email',
        'teen': 'trẻ',
        'style': 'phong cách',
        'topic': 'chủ đề',
        'file': 'tài liệu',
        'slides': 'tài liệu',
        'slide': 'tài liệu',
        'giáo trình': 'tài liệu',
        'slile': 'tài liệu',
        'silde': 'tài liệu',
        'web':'website',
        'online': 'website',
        'nope': 'không',
        'class': 'lớp học',
        'fitted': 'phù hợp',
        'chair': 'ghế',
        'table': 'bàn',
        'grammar': 'ngữ pháp',
        'speaking': 'kỹ năng nói',
        'listening': 'kỹ năng nghe',
        'listenning': 'kỹ năng nghe',
        'reading': 'kỹ năng đọc',
        'good': 'tốt',
        'english': 'tiếng anh',
        'bad': 'tệ',
        'feedback': 'phản hồi',
        'elab': 'lab',
        'group':'nhóm',
        'mic': 'microphone',
        'debate': 'thảo luận',
        'test': 'kiểm tra',
        'thưc': 'thực',
        'courses': 'khóa học',
        'funny': 'vui vẻ',
        'internet': 'mạng',
        'perfect': 'hoàn hảo',
        'quoa': 'qua',
        'thanks': 'cảm ơn',
        'thankss': 'cảm ơn',
        'ok': 'ổn',
        'sư phạm': 'giảng dạy'
        # Thêm các cặp ánh xạ khác nếu cần
    }
    for original, replacement in mapping_dict.items():
        # Sử dụng regex để đảm bảo chỉ thay thế các cụm từ chính xác
        pattern = r'\b' + re.escape(original) + r'\b'
        text = re.sub(pattern, replacement, text)

    # 4. Loại bỏ dấu câu
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 5. Loại bỏ số
    text = re.sub(r'\d+', '', text)

    # 6. Loại bỏ khoảng trắng thừa ở đầu và cuối
    text = text.strip()

    # 7. Chuẩn hóa văn bản
    text = text_normalize(text)

    # 8. Loại bỏ biểu tượng cảm xúc (emojis)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002500-\U00002BEF"  # Chinese characters
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # 9. Loại bỏ các tiền tố cụ thể nhưng giữ lại hậu tố
    text = re.sub(r'\bcolon(\w+)\b', r'\1', text)  # Loại bỏ tiền tố "colon" nhưng giữ lại hậu tố
    text = re.sub(r'\bdoubledot(\w+)\b', r'\1', text)  # Loại bỏ tiền tố "doubledot" nhưng giữ lại hậu tố

    # 10. Định nghĩa các tiền tố cần loại bỏ
    prefixes_to_remove = ['wzjwz', 'wwzjwz', 'dot']

    # 11. Loại bỏ các từ bắt đầu bằng bất kỳ tiền tố nào trong danh sách
    for prefix in prefixes_to_remove:
        text = re.sub(r'\b' + re.escape(prefix) + r'\w*\b', '', text)

    # 12. Loại bỏ các từ hiếm
    list_rare_words = [
        'fraction', 'altera', 'quad', 'cnpm', 'kaydotvn', 'daadotuitdotedudotvn', 'bohm', 'dotnet',
        'visual studio code', 'visual studio', 'visual code', 'visual', 'mutton quad', 'dreamweaver',
        'vertical fragmentation', 'i-ta-li-a', 'đbcl', 'crt', 'javabeans', 'if', 'for', 'while',
        'struct', 'pl', 'windows phone', 'pm', 'assembly', 'scrum_project', 'scrum project',
        'css', 'jquery', 'vector', 'gb', 'ram', 'ram', '...', 'codefun', 'egov', 'java', 'vmware',
        'placement', 'fork', 'round robin', 'patin', 'pattern', 'serverside', 'dotnet', 'c++',
        'javascript', 'ch', 'uit khoa', 'proteus', 'console', 'form', 'vật lý học', 'aep',
        'servlet', 'skype', 'doubledot', 'poster', 'everything', 'amp', 'app', 'seo', 'app',
        'progressive web', 'xim', 'win server', 'đối với', 'p', 's', 'p s', 'dropbox', 'bạn',
        'cplusplus', 'socket', 'sub', 'switch', 'tôi', 'em', 'các em', 'tụi em', 'chúng em', 'các bạn', 'tụi em', 'chúng em'
        'severside', 'network programing with csharp', 'dfd', 'naives bayes', 'naive', 'bayes', 'cs', 'js', 'max', 'elg', 'fix', 'proxy',
        'hub', 'bridge','switch', 'windows', 'turnitindotcom', 'extensive reading', 'reading', 'extensive', 'search', 'quick', 'file header','paper',
        'directx', 'windows', 'linux', 'vote', 'itdotf', 'router', 'silverlight', 'đa luồng', 'crack', 'wrede' ,'dbpedia','ontology', 'tmf', 'vhdl',
        'hdl', 'jsp', 'pđt', 'lisp', 'json', 'cpp', 'các_em', 'chúng_em', 'tụi_em', 'các_bạn', 'tụi_em', 'sinh_viên', 'là', 'và', 'thì', 'vì', 'mà', 'của', 'khi', 'như', 'lại', 'đó', 'đây', 'kia', 'ấy', 'sẽ', 'mình', 'nếu', 'vậy', 'rồi', 'với', 'bởi', 'mà', 'ấy', 'kia', 'sẽ', 'đó', 'dù', 'tuy', 'itp', 'forum', 'embeded', 'system', 'embeded system', 'embedded', 'titanium', 'blackbery', 'zun', 'phonegap', 'tizen', 'je', 'mediafire','toeic', 'ghz', 'cpu', 'module', 'datapath', 'papers', 'daa', 'dijktra', 'oracal', 'database', 'access', 'netbean', 'facebook', 'hackerrankdotcom', 'sort', 'multiagent', 'th', 'contemn', 'dbms', 'html', 'php', 'heapsort', 'khmtdotuitdotedudotvn', 'vdotv', 'engine', 'download','input', 'output', 'wtf','forum', 'poison', 'uit', 'career', 'như', 'version', 'outdoor', 'coursedotuitdotedudotvn', 'mini', 'matlab', 'standford', 'name', 'size', 'framework', 'ucla', 'comment', 'is','we','serverside', 'cassette', 'ios', 'android', 'scrum', 'itdote', 'xml', 'photo', 'down', 'unikey', '3dsmax', 'firmware', 'km','hackerrank', 'projectbase', 'er', 'gay', 'feed', 'mác – lênin', 'mác', 'lênin', 'coursedotuitdotedudotvn', 'nfc', 'chip', 'full', 'oi', 'ht', 'ubuntu', 'linux', 'it', 'wecode', 'code','oop', 'hướng đối tượng', 'cho','để','macbook', 'a_z', 'av', 'anh văn', 'đã','một','nhưng','học sinh'
    ]

    # Remove rare words using regex
    rarewords_pattern = r'\b(?:' + '|'.join(re.escape(word) for word in list_rare_words) + r')\b'
    text = re.sub(rarewords_pattern, '', text)

    # 13. Token hóa văn bản
    tokens = word_tokenize(text, format='text').split()
    tokens = [token for token in tokens if token not in stopwords_small]

    # 14. Loại bỏ các token rỗng nếu có
    tokens = [token for token in tokens if token.strip()]
    text = ' '.join(tokens)
    tokens = word_tokenize(text, format='text').split()
    return tokens


In [86]:
import string
import regex as re
import numpy as np
import pandas as pd
from underthesea import word_tokenize, text_normalize

# =======================
# Vietnamese Character Normalization Functions
# =======================

# Define Unicode and Unsigned Characters
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    """
    Load a dictionary mapping accented Vietnamese characters to their base forms.
    """
    dic = {}
    # Define the list of accented characters (1252 encoding)
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|' \
              'ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|' \
              'ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|' \
              'À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|' \
              'È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|' \
              'Ì|Í|Ỉ|Ĩ|Ị|' \
              'Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|' \
              'Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|' \
              'Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')

    # Define the list of UTF-8 encoded characters
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|" \
              "ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|" \
              "ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|" \
              "À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|" \
              "È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|" \
              "Ì|Í|Ỉ|Ĩ|Ị|" \
              "Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|" \
              "Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|" \
              "Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')

    # Create the dictionary mapping
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

dicchar = loaddicchar()

def covert_unicode(txt):
    """
    Convert accented Vietnamese characters to their base forms.
    """
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|' \
        r'è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|' \
        r'ì|í|ỉ|ĩ|ị|' \
        r'ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|' \
        r'ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|' \
        r'ỳ|ý|ỷ|ỹ|ỵ|' \
        r'À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|' \
        r'È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|' \
        r'Ì|Í|Ỉ|Ĩ|Ị|' \
        r'Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|' \
        r'Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|' \
        r'Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# =======================
# Existing Vietnamese Accent Normalization Functions
# =======================

# Define mappings for Vietnamese vowels and their accents
# Note: Ensure that bang_nguyen_am and nguyen_am_to_ids are fully defined
bang_nguyen_am = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ'],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ'],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị'],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ'],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ'],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ'],
    # Add more vowels if necessary
]

# Create a mapping from vowel to its indices in bang_nguyen_am
nguyen_am_to_ids = {}
for idx, vowel_group in enumerate(bang_nguyen_am):
    for jdx, vowel in enumerate(vowel_group):
        nguyen_am_to_ids[vowel] = (idx, jdx)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
            if not qu_or_gi or index != 1:
                nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1], (-1, -1))
                if x != -1:
                    chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids.get(chars[index], (-1, -1))
        if x == 4 or x == 8:  # Example indices for specific vowels like 'ê', 'ơ'
            chars[index] = bang_nguyen_am[x][dau_cau]
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[0]], (-1, -1))
            if x != -1:
                chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[1]], (-1, -1))
            if x != -1:
                chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[1]], (-1, -1))
        if x != -1:
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
    Normalize Vietnamese sentence to the old typing standard.
    :param sentence: str, input Vietnamese sentence
    :return: str, normalized sentence
    """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        # Using regex to handle punctuation around words
        cw = re.sub(r'(^\p{P}*)([\p{L}]+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tieng_viet(cw[1])
            words[index] = ''.join(cw)
    return ' '.join(words)

# =======================
# Additional Unicode Normalization Function
# =======================

# The covert_unicode function replaces accented characters with their base forms
# This can be used before or after the existing normalization depending on your needs

# =======================
# Existing Text Preprocessing Functions
# =======================

# Load stopwords from CSV
filename = 'vietnamese-stopwords.csv'
data = pd.read_csv(filename, names=['word'])
list_stopwords = data['word'].tolist()  # Convert to list for easier checking
myarray = np.asarray(list_stopwords)

# Load additional stopwords from text file
stopwords_small = []
with open('stopwords-vi_news.txt', 'r', encoding='utf-8') as file:
    stopwords_small = file.readlines()

# Clean stopwords list
stopwords_small = [line.strip().replace(" ", "_") for line in stopwords_small]

list_rare_words_1 = [
        'fraction', 'altera', 'quad', 'cnpm', 'kaydotvn', 'daadotuitdotedudotvn', 'bohm', 'dotnet',
        'visual studio code', 'visual studio', 'visual code', 'visual', 'mutton quad', 'dreamweaver',
        'vertical fragmentation', 'i-ta-li-a', 'đbcl', 'crt', 'javabeans', 'if', 'for', 'while',
        'struct', 'pl', 'windows phone', 'pm', 'assembly', 'scrum_project', 'scrum project',
        'css', 'jquery', 'vector', 'gb', 'ram', 'ram', '...', 'codefun', 'egov', 'java', 'vmware',
        'placement', 'fork', 'round robin', 'patin', 'pattern', 'serverside', 'dotnet', 'c++',
        'javascript', 'ch', 'uit khoa', 'proteus', 'console', 'form', 'vật lý học', 'aep',
        'servlet', 'skype', 'doubledot', 'poster', 'everything', 'amp', 'app', 'seo', 'app',
        'progressive web', 'xim', 'win server', 'đối với', 'p', 's', 'p s', 'dropbox', 'bạn',
        'cplusplus', 'socket', 'sub', 'switch', 'tôi', 'em', 'các em', 'tụi em', 'chúng em', 'các bạn', 'tụi em', 'chúng em'
        'severside', 'network programing with csharp', 'dfd', 'naives bayes', 'naive', 'bayes', 'cs', 'js', 'max', 'elg', 'fix', 'proxy',
        'hub', 'bridge','switch', 'windows', 'turnitindotcom', 'extensive reading', 'reading', 'extensive', 'search', 'quick', 'file header','paper',
        'directx', 'windows', 'linux', 'vote', 'itdotf', 'router', 'silverlight', 'đa luồng', 'crack', 'wrede' ,'dbpedia','ontology', 'tmf', 'vhdl',
        'hdl', 'jsp', 'pđt', 'lisp', 'json', 'cpp', 'các_em', 'chúng_em', 'tụi_em', 'các_bạn', 'tụi_em', 'sinh_viên', 'là', 'và', 'thì', 'vì', 'mà', 'của', 'khi', 'như', 'lại', 'đó', 'đây', 'kia', 'ấy', 'sẽ', 'mình', 'nếu', 'vậy', 'rồi', 'với', 'bởi', 'mà', 'ấy', 'kia', 'sẽ', 'đó', 'dù', 'tuy', 'itp', 'forum', 'embeded', 'system', 'embeded system', 'embedded', 'titanium', 'blackbery', 'zun', 'phonegap', 'tizen', 'je', 'mediafire','toeic', 'ghz', 'cpu', 'module', 'datapath', 'papers', 'daa', 'dijktra', 'oracal', 'database', 'access', 'netbean', 'facebook', 'hackerrankdotcom', 'sort', 'multiagent', 'th', 'contemn', 'dbms', 'html', 'php', 'heapsort', 'khmtdotuitdotedudotvn', 'vdotv', 'engine', 'download','input', 'output', 'wtf','forum', 'poison', 'uit', 'career', 'như', 'version', 'outdoor', 'coursedotuitdotedudotvn', 'mini', 'matlab', 'standford', 'name', 'size', 'framework', 'ucla', 'comment', 'is','we','serverside', 'cassette', 'ios', 'android', 'scrum', 'itdote', 'xml', 'photo', 'down', 'unikey', '3dsmax', 'firmware', 'km','hackerrank', 'projectbase', 'er', 'gay', 'feed', 'mác – lênin', 'mác', 'lênin', 'coursedotuitdotedudotvn', 'nfc', 'chip', 'full', 'oi', 'ht', 'ubuntu', 'linux', 'it', 'wecode', 'code','oop', 'hướng đối tượng', 'cho','để','macbook', 'a_z', 'av', 'anh văn', 'đã','một','nhưng','học sinh'
    ]
# Clean stopwords list
stopwords_small = [line.strip().replace(" ", "_") for line in stopwords_small]
stopwords_small.append('dot')
stopwords_small.append('giảng_viên')
for item in list_rare_words_1:
    stopwords_small.append(item)

def preprocess_text_vietnamese_to_text(text):
    """
    Preprocess Vietnamese text by normalizing accents, removing unwanted elements,
    and tokenizing.
    :param text: str, input Vietnamese text
    :return: str, processed text
    """
    # 0. Normalize Unicode accents (remove diacritics)
    text = covert_unicode(text)

    # 1. Normalize Vietnamese accents using existing functions
    text = chuan_hoa_dau_cau_tieng_viet(text)

    # 2. Chuyển đổi văn bản thành chữ thường
    text = text.lower()

    # 3. Thực hiện ánh xạ các cụm từ theo từ điển
    mapping_dict = {
        "thầy giáo": "giảng viên",
        "cô giáo": "giảng viên",
        "thầy": "giảng viên",
        "cô": "giảng viên",
        "giáo viên": "giảng viên",
        'vói': 'với',
        'giời': 'giờ',
        'nhiệt hình': 'nhiệt tình',
        'side': 'slide',
        'tân tình': 'tận tình',
        'teacher': 'giảng viên',
        'sadcolon': 'colonsad',
        'vi dụ': 'ví dụ',
        'easy': 'dễ',
        'so vời': 'so với',
        'tâp': 'tập',
        'av': 'tiếng anh',
        'nhannh': 'nhanh',
        'h': 'giờ',
        'đc': 'được',
        'dc': 'được',
        'smilesmile': 'smile',
        'tron': 'trong',
        'thướng': 'hướng',
        'nghung': 'nhưng',
        'chon': 'chọn',
        'them': 'thêm',
        'day': 'dạy',
        'midterm': 'giữa kỳ',
        'vi': 'vì',
        'quýêt': 'quyết',
        'teamwork': 'làm việc nhóm',
        'over time': 'quá giờ',
        'overtime': 'quá giờ',
        'ot': 'quá giờ',
        'outcome': 'mục tiêu',
        'basic': 'cơ bản',
        'check': 'kiểm tra',
        'gmail': 'email',
        'mail': 'email',
        'teen': 'trẻ',
        'style': 'phong cách',
        'topic': 'chủ đề',
        'file': 'tài liệu',
        'slides': 'tài liệu',
        'slide': 'tài liệu',
        'giáo trình': 'tài liệu',
        'slile': 'tài liệu',
        'silde': 'tài liệu',
        'web':'website',
        'online': 'website',
        'nope': 'không',
        'class': 'lớp học',
        'fitted': 'phù hợp',
        'chair': 'ghế',
        'table': 'bàn',
        'grammar': 'ngữ pháp',
        'speaking': 'kỹ năng nói',
        'listening': 'kỹ năng nghe',
        'listenning': 'kỹ năng nghe',
        'reading': 'kỹ năng đọc',
        'good': 'tốt',
        'english': 'tiếng anh',
        'bad': 'tệ',
        'feedback': 'phản hồi',
        'elab': 'lab',
        'group':'nhóm',
        'mic': 'microphone',
        'debate': 'thảo luận',
        'test': 'kiểm tra',
        'thưc': 'thực',
        'courses': 'khóa học',
        'funny': 'vui vẻ',
        'internet': 'mạng',
        'perfect': 'hoàn hảo',
        'quoa': 'qua',
        'thanks': 'cảm ơn',
        'thankss': 'cảm ơn'
        # Thêm các cặp ánh xạ khác nếu cần
    }
    for original, replacement in mapping_dict.items():
        # Sử dụng regex để đảm bảo chỉ thay thế các cụm từ chính xác
        pattern = r'\b' + re.escape(original) + r'\b'
        text = re.sub(pattern, replacement, text)

    # 4. Loại bỏ dấu câu
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 5. Loại bỏ số
    text = re.sub(r'\d+', '', text)

    # 6. Loại bỏ khoảng trắng thừa ở đầu và cuối
    text = text.strip()

    # 7. Chuẩn hóa văn bản
    text = text_normalize(text)

    # 8. Loại bỏ biểu tượng cảm xúc (emojis)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002500-\U00002BEF"  # Chinese characters
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # 9. Loại bỏ các tiền tố cụ thể nhưng giữ lại hậu tố
    text = re.sub(r'\bcolon(\w+)\b', r'\1', text)  # Loại bỏ tiền tố "colon" nhưng giữ lại hậu tố
    text = re.sub(r'\bdoubledot(\w+)\b', r'\1', text)  # Loại bỏ tiền tố "doubledot" nhưng giữ lại hậu tố
    # 10. Định nghĩa các tiền tố cần loại bỏ
    prefixes_to_remove = ['wzjwz', 'wwzjwz', 'dot']
    text = re.sub(r'\b\w*smile\w*\b', 'smile', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*surprise\w*\b', 'surprise', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*sad\w*\b', 'sad', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*contemn\w*\b', 'contemn', text, flags=re.IGNORECASE)

    # 11. Loại bỏ các từ bắt đầu bằng bất kỳ tiền tố nào trong danh sách
    for prefix in prefixes_to_remove:
        text = re.sub(r'\b' + re.escape(prefix) + r'\w*\b', '', text)

    # 12. Loại bỏ các từ hiếm
    list_rare_words = [
        'fraction', 'altera', 'quad', 'cnpm', 'kaydotvn', 'daadotuitdotedudotvn', 'bohm', 'dotnet',
        'visual studio code', 'visual studio', 'visual code', 'visual', 'mutton quad', 'dreamweaver',
        'vertical fragmentation', 'i-ta-li-a', 'đbcl', 'crt', 'javabeans', 'if', 'for', 'while',
        'struct', 'pl', 'windows phone', 'pm', 'assembly', 'scrum_project', 'scrum project',
        'css', 'jquery', 'vector', 'gb', 'ram', 'ram', '...', 'codefun', 'egov', 'java', 'vmware',
        'placement', 'fork', 'round robin', 'patin', 'pattern', 'serverside', 'dotnet', 'c++',
        'javascript', 'ch', 'uit khoa', 'proteus', 'console', 'form', 'vật lý học', 'aep',
        'servlet', 'skype', 'doubledot', 'poster', 'everything', 'amp', 'app', 'seo', 'app',
        'progressive web', 'xim', 'win server', 'đối với', 'p', 's', 'p s', 'dropbox', 'bạn',
        'cplusplus', 'socket', 'sub', 'switch', 'tôi', 'em', 'các em', 'tụi em', 'chúng em', 'các bạn', 'tụi em', 'chúng em'
        'severside', 'network programing with csharp', 'dfd', 'naives bayes', 'naive', 'bayes', 'cs', 'js', 'max', 'elg', 'fix', 'proxy',
        'hub', 'bridge','switch', 'windows', 'turnitindotcom', 'extensive reading', 'reading', 'extensive', 'search', 'quick', 'file header','paper',
        'directx', 'windows', 'linux', 'vote', 'itdotf', 'router', 'silverlight', 'đa luồng', 'crack', 'wrede' ,'dbpedia','ontology', 'tmf', 'vhdl',
        'hdl', 'jsp', 'pđt', 'lisp', 'json', 'cpp', 'các_em', 'chúng_em', 'tụi_em', 'các_bạn', 'tụi_em', 'sinh_viên', 'là', 'và', 'thì', 'vì', 'mà', 'của', 'khi', 'như', 'lại', 'đó', 'đây', 'kia', 'ấy', 'sẽ', 'mình', 'nếu', 'vậy', 'rồi', 'với', 'bởi', 'mà', 'ấy', 'kia', 'sẽ', 'đó', 'dù', 'tuy', 'itp', 'forum', 'embeded', 'system', 'embeded system', 'embedded', 'titanium', 'blackbery', 'zun', 'phonegap', 'tizen', 'je', 'mediafire','toeic', 'ghz', 'cpu', 'module', 'datapath', 'papers', 'daa', 'dijktra', 'oracal', 'database', 'access', 'netbean', 'facebook', 'hackerrankdotcom', 'sort', 'multiagent', 'th', 'contemn', 'dbms', 'html', 'php', 'heapsort', 'khmtdotuitdotedudotvn', 'vdotv', 'engine', 'download','input', 'output', 'wtf','forum', 'poison', 'uit', 'career', 'như', 'version', 'outdoor', 'coursedotuitdotedudotvn', 'mini', 'matlab', 'standford', 'name', 'size', 'framework', 'ucla', 'comment', 'is','we','serverside', 'cassette', 'ios', 'android', 'scrum', 'itdote', 'xml', 'photo', 'down', 'unikey', '3dsmax', 'firmware', 'km','hackerrank', 'projectbase', 'er', 'gay', 'feed', 'mác – lênin', 'mác', 'lênin', 'coursedotuitdotedudotvn', 'nfc', 'chip', 'full', 'oi', 'ht', 'ubuntu', 'linux', 'it', 'wecode', 'code','oop', 'hướng đối tượng', 'cho','để','macbook', 'a_z', 'học sinh'
    ]

    # Remove rare words using regex
    rarewords_pattern = r'\b(?:' + '|'.join(re.escape(word) for word in list_rare_words) + r')\b'
    text = re.sub(rarewords_pattern, '', text)

    # 13. Token hóa văn bản
    tokens = word_tokenize(text, format='text').split()
    tokens = [token for token in tokens if token not in stopwords_small]

    # 14. Loại bỏ các token rỗng nếu có
    tokens = [token for token in tokens if token.strip()]

    # 15. Ghép các token lại thành chuỗi văn bản đã xử lý
    text = ' '.join(tokens)
    tokens = word_tokenize(text, format='text').split()
    processed_text = ' '.join(tokens)
    return processed_text


In [87]:
import string
import regex as re
import numpy as np
import pandas as pd
from underthesea import word_tokenize, text_normalize

# =======================
# Vietnamese Character Normalization Functions
# =======================

# Define Unicode and Unsigned Characters
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    """
    Load a dictionary mapping accented Vietnamese characters to their base forms.
    """
    dic = {}
    # Define the list of accented characters (1252 encoding)
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|' \
              'ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|' \
              'ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|' \
              'À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|' \
              'È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|' \
              'Ì|Í|Ỉ|Ĩ|Ị|' \
              'Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|' \
              'Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|' \
              'Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')

    # Define the list of UTF-8 encoded characters
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|" \
              "ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|" \
              "ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|" \
              "À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|" \
              "È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|" \
              "Ì|Í|Ỉ|Ĩ|Ị|" \
              "Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|" \
              "Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|" \
              "Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')

    # Create the dictionary mapping
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

dicchar = loaddicchar()

def covert_unicode(txt):
    """
    Convert accented Vietnamese characters to their base forms.
    """
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|' \
        r'è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|' \
        r'ì|í|ỉ|ĩ|ị|' \
        r'ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|' \
        r'ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|' \
        r'ỳ|ý|ỷ|ỹ|ỵ|' \
        r'À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|' \
        r'È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|' \
        r'Ì|Í|Ỉ|Ĩ|Ị|' \
        r'Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|' \
        r'Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|' \
        r'Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# =======================
# Existing Vietnamese Accent Normalization Functions
# =======================

# Define mappings for Vietnamese vowels and their accents
# Note: Ensure that bang_nguyen_am and nguyen_am_to_ids are fully defined
bang_nguyen_am = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ'],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ'],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị'],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ'],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ'],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ'],
    # Add more vowels if necessary
]

# Create a mapping from vowel to its indices in bang_nguyen_am
nguyen_am_to_ids = {}
for idx, vowel_group in enumerate(bang_nguyen_am):
    for jdx, vowel in enumerate(vowel_group):
        nguyen_am_to_ids[vowel] = (idx, jdx)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
            if not qu_or_gi or index != 1:
                nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1], (-1, -1))
                if x != -1:
                    chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids.get(chars[index], (-1, -1))
        if x == 4 or x == 8:  # Example indices for specific vowels like 'ê', 'ơ'
            chars[index] = bang_nguyen_am[x][dau_cau]
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[0]], (-1, -1))
            if x != -1:
                chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[1]], (-1, -1))
            if x != -1:
                chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[1]], (-1, -1))
        if x != -1:
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
    Normalize Vietnamese sentence to the old typing standard.
    :param sentence: str, input Vietnamese sentence
    :return: str, normalized sentence
    """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        # Using regex to handle punctuation around words
        cw = re.sub(r'(^\p{P}*)([\p{L}]+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tieng_viet(cw[1])
            words[index] = ''.join(cw)
    return ' '.join(words)

# =======================
# Additional Unicode Normalization Function
# =======================

# The covert_unicode function replaces accented characters with their base forms
# This can be used before or after the existing normalization depending on your needs

# =======================
# Existing Text Preprocessing Functions
# =======================

# Load stopwords from CSV
filename = 'vietnamese-stopwords.csv'
data = pd.read_csv(filename, names=['word'])
list_stopwords = data['word'].tolist()  # Convert to list for easier checking
myarray = np.asarray(list_stopwords)

# Load additional stopwords from text file
stopwords_small = []
with open('stopwords-vi_news.txt', 'r', encoding='utf-8') as file:
    stopwords_small = file.readlines()

# Clean stopwords list
stopwords_small = [line.strip().replace(" ", "_") for line in stopwords_small]

list_rare_words_1 = [
        'fraction', 'altera', 'quad', 'cnpm', 'kaydotvn', 'daadotuitdotedudotvn', 'bohm', 'dotnet',
        'visual studio code', 'visual studio', 'visual code', 'visual', 'mutton quad', 'dreamweaver',
        'vertical fragmentation', 'i-ta-li-a', 'đbcl', 'crt', 'javabeans', 'if', 'for', 'while',
        'struct', 'pl', 'windows phone', 'pm', 'assembly', 'scrum_project', 'scrum project',
        'css', 'jquery', 'vector', 'gb', 'ram', 'ram', '...', 'codefun', 'egov', 'java', 'vmware',
        'placement', 'fork', 'round robin', 'patin', 'pattern', 'serverside', 'dotnet', 'c++',
        'javascript', 'ch', 'uit khoa', 'proteus', 'console', 'form', 'vật lý học', 'aep',
        'servlet', 'skype', 'doubledot', 'poster', 'everything', 'amp', 'app', 'seo', 'app',
        'progressive web', 'xim', 'win server', 'p', 's', 'p s', 'dropbox',
        'cplusplus', 'socket', 'sub', 'switch',
        'severside', 'network programing with csharp', 'dfd', 'naives bayes', 'naive', 'bayes', 'cs', 'js', 'max', 'elg', 'fix', 'proxy',
        'hub', 'bridge','switch', 'windows', 'turnitindotcom', 'extensive reading', 'reading', 'extensive', 'search', 'quick', 'file header','paper',
        'directx', 'windows', 'linux', 'vote', 'itdotf', 'router', 'silverlight', 'đa luồng', 'crack', 'wrede' ,'dbpedia','ontology', 'tmf', 'vhdl',
        'hdl', 'jsp', 'pđt', 'lisp', 'json', 'cpp', 'itp', 'forum', 'embeded', 'system', 'embeded system', 'embedded', 'titanium', 'blackbery', 'zun', 'phonegap', 'tizen', 'je', 'mediafire','toeic', 'ghz', 'cpu', 'module', 'datapath', 'papers', 'daa', 'dijktra', 'oracal', 'database', 'access', 'netbean', 'facebook', 'hackerrankdotcom', 'sort', 'multiagent', 'th', 'contemn', 'dbms', 'html', 'php', 'heapsort', 'khmtdotuitdotedudotvn', 'vdotv', 'engine', 'download','input', 'output','forum', 'poison', 'uit', 'career', 'version', 'outdoor', 'coursedotuitdotedudotvn', 'mini', 'matlab', 'standford', 'name', 'size', 'framework', 'ucla', 'comment', 'is','we','serverside', 'cassette', 'ios', 'android', 'scrum', 'itdote', 'xml', 'photo', 'down', 'unikey', '3dsmax', 'firmware', 'km','hackerrank', 'projectbase', 'er', 'gay', 'feed', 'mác – lênin', 'mác', 'lênin', 'coursedotuitdotedudotvn', 'nfc', 'chip', 'full', 'oi', 'ht', 'ubuntu', 'linux', 'it', 'wecode','oop', 'hướng đối tượng','macbook', 'a_z', 'môn_toán', 'môn_lý'
    ]
# Clean stopwords list
stopwords_small = [line.strip().replace(" ", "_") for line in stopwords_small]
stopwords_small.append('dot')
# stopwords_small.append('giảng_viên')
for item in list_rare_words_1:
    stopwords_small.append(item)

def preprocess_text_vietnamese_to_text_simple(text):
    """
    Preprocess Vietnamese text by normalizing accents, removing unwanted elements,
    and tokenizing.
    :param text: str, input Vietnamese text
    :return: str, processed text
    """
    # 0. Normalize Unicode accents (remove diacritics)
    text = covert_unicode(text)

    # 1. Normalize Vietnamese accents using existing functions
    text = chuan_hoa_dau_cau_tieng_viet(text)

    # 2. Chuyển đổi văn bản thành chữ thường
    text = text.lower()

    # 3. Thực hiện ánh xạ các cụm từ theo từ điển
    mapping_dict = {
        "thầy giáo": "giảng viên",
        "cô giáo": "giảng viên",
        "thầy": "giảng viên",
        "cô": "giảng viên",
        "giáo viên": "giảng viên",
        'vói': 'với',
        'giời': 'giờ',
        'nhiệt hình': 'nhiệt tình',
        'side': 'slide',
        'tân tình': 'tận tình',
        'teacher': 'giảng viên',
        'sadcolon': 'colonsad',
        'vi dụ': 'ví dụ',
        'easy': 'dễ',
        'so vời': 'so với',
        'tâp': 'tập',
        'av': 'tiếng anh',
        'nhannh': 'nhanh',
        'h': 'giờ',
        'đc': 'được',
        'dc': 'được',
        'smilesmile': 'smile',
        'tron': 'trong',
        'thướng': 'hướng',
        'nghung': 'nhưng',
        'chon': 'chọn',
        'them': 'thêm',
        'day': 'giảng dạy',
        'midterm': 'giữa kỳ',
        'vi': 'vì',
        'quýêt': 'quyết',
        'teamwork': 'làm việc nhóm',
        'over time': 'quá giờ',
        'overtime': 'quá giờ',
        'ot': 'quá giờ',
        'outcome': 'mục tiêu',
        'basic': 'cơ bản',
        'check': 'kiểm tra',
        'gmail': 'email',
        'mail': 'email',
        'teen': 'trẻ',
        'style': 'phong cách',
        'topic': 'chủ đề',
        'file': 'tài liệu',
        'slides': 'tài liệu',
        'slide': 'tài liệu',
        'giáo trình': 'tài liệu',
        'slile': 'tài liệu',
        'silde': 'tài liệu',
        'web':'website',
        'online': 'website',
        'nope': 'không',
        'class': 'lớp học',
        'fitted': 'phù hợp',
        'chair': 'ghế',
        'table': 'bàn',
        'grammar': 'ngữ pháp',
        'speaking': 'kỹ năng nói',
        'listening': 'kỹ năng nghe',
        'listenning': 'kỹ năng nghe',
        'reading': 'kỹ năng đọc',
        'good': 'tốt',
        'english': 'tiếng anh',
        'bad': 'tệ',
        'feedback': 'phản hồi',
        'elab': 'lab',
        'group':'nhóm',
        'mic': 'microphone',
        'debate': 'thảo luận',
        'test': 'kiểm tra',
        'thưc': 'thực',
        'courses': 'khóa học',
        'funny': 'vui vẻ',
        'internet': 'mạng',
        'perfect': 'hoàn hảo',
        'quoa': 'qua',
        'thanks': 'cảm ơn',
        'thankss': 'cảm ơn',
        'dạy':'giảng dạy'

        # Thêm các cặp ánh xạ khác nếu cần
    }
    for original, replacement in mapping_dict.items():
        # Sử dụng regex để đảm bảo chỉ thay thế các cụm từ chính xác
        pattern = r'\b' + re.escape(original) + r'\b'
        text = re.sub(pattern, replacement, text)

    # 4. Loại bỏ dấu câu
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 5. Loại bỏ số
    text = re.sub(r'\d+', '', text)

    # 6. Loại bỏ khoảng trắng thừa ở đầu và cuối
    text = text.strip()

    # 7. Chuẩn hóa văn bản
    text = text_normalize(text)

    # 8. Loại bỏ biểu tượng cảm xúc (emojis)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002500-\U00002BEF"  # Chinese characters
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # 9. Loại bỏ các tiền tố cụ thể nhưng giữ lại hậu tố
    text = re.sub(r'\bcolon(\w+)\b', r'\1', text)  # Loại bỏ tiền tố "colon" nhưng giữ lại hậu tố
    text = re.sub(r'\bdoubledot(\w+)\b', r'\1', text)  # Loại bỏ tiền tố "doubledot" nhưng giữ lại hậu tố
    # 10. Định nghĩa các tiền tố cần loại bỏ
    prefixes_to_remove = ['wzjwz', 'wwzjwz']
    text = re.sub(r'\b\w*smile\w*\b', 'smile', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*surprise\w*\b', 'surprise', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*sad\w*\b', 'sad', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*contemn\w*\b', 'contemn', text, flags=re.IGNORECASE)

    # 11. Loại bỏ các từ bắt đầu bằng bất kỳ tiền tố nào trong danh sách
    for prefix in prefixes_to_remove:
        text = re.sub(r'\b' + re.escape(prefix) + r'\w*\b', '', text)

    # 12. Loại bỏ các từ hiếm
    list_rare_words = [
        'fraction', 'altera', 'quad', 'cnpm', 'kaydotvn', 'daadotuitdotedudotvn', 'bohm', 'dotnet',
        'visual studio code', 'visual studio', 'visual code', 'visual', 'mutton quad', 'dreamweaver',
        'vertical fragmentation', 'i-ta-li-a', 'đbcl', 'crt', 'javabeans', 'if', 'for', 'while',
        'struct', 'pl', 'windows phone', 'pm', 'assembly', 'scrum_project', 'scrum project',
        'css', 'jquery', 'vector', 'gb', 'ram', 'ram', '...', 'codefun', 'egov', 'java', 'vmware',
        'placement', 'fork', 'round robin', 'patin', 'pattern', 'serverside', 'dotnet', 'c++',
        'javascript', 'ch', 'uit khoa', 'proteus', 'console', 'form', 'vật lý học', 'aep',
        'servlet', 'skype', 'doubledot', 'poster', 'everything', 'amp', 'app', 'seo', 'app',
        'progressive web', 'xim', 'win server', 'p', 's', 'p s', 'dropbox',
        'cplusplus', 'socket', 'sub', 'switch',
        'severside', 'network programing with csharp', 'dfd', 'naives bayes', 'naive', 'bayes', 'cs', 'js', 'max', 'elg', 'fix', 'proxy',
        'hub', 'bridge','switch', 'windows', 'turnitindotcom', 'extensive reading', 'reading', 'extensive', 'search', 'quick', 'file header','paper',
        'directx', 'windows', 'linux', 'vote', 'itdotf', 'router', 'silverlight', 'đa luồng', 'crack', 'wrede' ,'dbpedia','ontology', 'tmf', 'vhdl',
        'hdl', 'jsp', 'pđt', 'lisp', 'json', 'cpp', 'itp', 'forum', 'embeded', 'system', 'embeded system', 'embedded', 'titanium', 'blackbery', 'zun', 'phonegap', 'tizen', 'je', 'mediafire','toeic', 'ghz', 'cpu', 'module', 'datapath', 'papers', 'daa', 'dijktra', 'oracal', 'database', 'access', 'netbean', 'facebook', 'hackerrankdotcom', 'sort', 'multiagent', 'th', 'contemn', 'dbms', 'html', 'php', 'heapsort', 'khmtdotuitdotedudotvn', 'vdotv', 'engine', 'download','input', 'output', 'wtf','forum', 'poison', 'uit', 'career', 'version', 'outdoor', 'coursedotuitdotedudotvn', 'mini', 'matlab', 'standford', 'name', 'size', 'framework', 'ucla', 'comment', 'is','we','serverside', 'cassette', 'ios', 'android', 'scrum', 'itdote', 'xml', 'photo', 'down', 'unikey', '3dsmax', 'firmware', 'km','hackerrank', 'projectbase', 'er', 'gay', 'feed', 'mác – lênin', 'mác', 'lênin', 'coursedotuitdotedudotvn', 'nfc', 'chip', 'full', 'oi', 'ht', 'ubuntu', 'linux', 'it', 'wecode','oop', 'hướng đối tượng','macbook', 'a_z', 'môn_toán', 'môn_lý'
    ]
    # Remove rare words using regex
    rarewords_pattern = r'\b(?:' + '|'.join(re.escape(word) for word in list_rare_words) + r')\b'
    text = re.sub(rarewords_pattern, '', text)

    # 13. Token hóa văn bản
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stopwords_small]

    list_ai_stopwords = [
        'và', 'cho', 'với', 'của', 'các', 'để', 'trong', 'về', 'thì', 'trên', 'vào',
        'một', 'ra', 'này', 'lại', 'mà', 'như', 'gì', 'cũng', 'lên', 'chỉ', 'đến',
        'khi', 'những', 'phòng', 'phần', 'buổi', 'đi', 'việc', 'là', 'có', 'nên',
        'môn', 'thêm', 'còn', 'nhưng', 'cách', 'giờ', 'thời_gian', 'nội_dung',
        'luôn', 'ít', 'vì', 'tiếng', 'anh', 'vấn_đề', 'phải', 'đó', 'có_thể', 'hay', 'được'
    ]

    tokens = [token for token in tokens if token not in list_ai_stopwords]

    # 14. Loại bỏ các token rỗng nếu có
    tokens = [token for token in tokens if token.strip()]

    # 15. Ghép các token lại thành chuỗi văn bản đã xử lý
    text = ' '.join(tokens)
    tokens = word_tokenize(text, format='text').split()
    processed_text = ' '.join(tokens)
    return processed_text


In [88]:
import string
import regex as re
import numpy as np
import pandas as pd
from underthesea import word_tokenize, text_normalize

# =======================
# Vietnamese Character Normalization Functions
# =======================

# Define Unicode and Unsigned Characters
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    """
    Load a dictionary mapping accented Vietnamese characters to their base forms.
    """
    dic = {}
    # Define the list of accented characters (1252 encoding)
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|' \
              'ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|' \
              'ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|' \
              'À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|' \
              'È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|' \
              'Ì|Í|Ỉ|Ĩ|Ị|' \
              'Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|' \
              'Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|' \
              'Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split('|')

    # Define the list of UTF-8 encoded characters
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|" \
              "ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|" \
              "ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|" \
              "À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|" \
              "È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|" \
              "Ì|Í|Ỉ|Ĩ|Ị|" \
              "Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|" \
              "Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|" \
              "Ỳ|Ý|Ỷ|Ỹ|Ỵ".split('|')

    # Create the dictionary mapping
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

dicchar = loaddicchar()

def covert_unicode(txt):
    """
    Convert accented Vietnamese characters to their base forms.
    """
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|' \
        r'è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|' \
        r'ì|í|ỉ|ĩ|ị|' \
        r'ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|' \
        r'ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|' \
        r'ỳ|ý|ỷ|ỹ|ỵ|' \
        r'À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|' \
        r'È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|' \
        r'Ì|Í|Ỉ|Ĩ|Ị|' \
        r'Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|' \
        r'Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|' \
        r'Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# =======================
# Existing Vietnamese Accent Normalization Functions
# =======================

# Define mappings for Vietnamese vowels and their accents
# Note: Ensure that bang_nguyen_am and nguyen_am_to_ids are fully defined
bang_nguyen_am = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ'],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ'],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị'],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ'],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ'],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ'],
    # Add more vowels if necessary
]

# Create a mapping from vowel to its indices in bang_nguyen_am
nguyen_am_to_ids = {}
for idx, vowel_group in enumerate(bang_nguyen_am):
    for jdx, vowel in enumerate(vowel_group):
        nguyen_am_to_ids[vowel] = (idx, jdx)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
            if not qu_or_gi or index != 1:
                nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1], (-1, -1))
                if x != -1:
                    chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids.get(chars[index], (-1, -1))
        if x == 4 or x == 8:  # Example indices for specific vowels like 'ê', 'ơ'
            chars[index] = bang_nguyen_am[x][dau_cau]
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[0]], (-1, -1))
            if x != -1:
                chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[1]], (-1, -1))
            if x != -1:
                chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids.get(chars[nguyen_am_index[1]], (-1, -1))
        if x != -1:
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
    Normalize Vietnamese sentence to the old typing standard.
    :param sentence: str, input Vietnamese sentence
    :return: str, normalized sentence
    """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        # Using regex to handle punctuation around words
        cw = re.sub(r'(^\p{P}*)([\p{L}]+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tieng_viet(cw[1])
            words[index] = ''.join(cw)
    return ' '.join(words)

# =======================
# Additional Unicode Normalization Function
# =======================

# The covert_unicode function replaces accented characters with their base forms
# This can be used before or after the existing normalization depending on your needs

# =======================
# Existing Text Preprocessing Functions
# =======================

# Load stopwords from CSV
filename = 'vietnamese-stopwords.csv'
data = pd.read_csv(filename, names=['word'])
list_stopwords = data['word'].tolist()  # Convert to list for easier checking
myarray = np.asarray(list_stopwords)

# Load additional stopwords from text file
stopwords_small = []
with open('stopwords-vi_news.txt', 'r', encoding='utf-8') as file:
    stopwords_small = file.readlines()

# Clean stopwords list
stopwords_small = [line.strip().replace(" ", "_") for line in stopwords_small]

list_rare_words_1 = [
        'fraction', 'altera', 'quad', 'cnpm', 'kaydotvn', 'daadotuitdotedudotvn', 'bohm', 'dotnet',
        'visual studio code', 'visual studio', 'visual code', 'visual', 'mutton quad', 'dreamweaver',
        'vertical fragmentation', 'i-ta-li-a', 'đbcl', 'crt', 'javabeans', 'if', 'for', 'while',
        'struct', 'pl', 'windows phone', 'pm', 'assembly', 'scrum_project', 'scrum project',
        'css', 'jquery', 'vector', 'gb', 'ram', 'ram', '...', 'codefun', 'egov', 'java', 'vmware',
        'placement', 'fork', 'round robin', 'patin', 'pattern', 'serverside', 'dotnet', 'c++',
        'javascript', 'ch', 'uit khoa', 'proteus', 'console', 'form', 'vật lý học', 'aep',
        'servlet', 'skype', 'doubledot', 'poster', 'everything', 'amp', 'app', 'seo', 'app',
        'progressive web', 'xim', 'win server', 'p', 's', 'p s', 'dropbox',
        'cplusplus', 'socket', 'sub', 'switch',
        'severside', 'network programing with csharp', 'dfd', 'naives bayes', 'naive', 'bayes', 'cs', 'js', 'max', 'elg', 'fix', 'proxy',
        'hub', 'bridge','switch', 'windows', 'turnitindotcom', 'extensive reading', 'reading', 'extensive', 'search', 'quick', 'file header','paper',
        'directx', 'windows', 'linux', 'vote', 'itdotf', 'router', 'silverlight', 'đa luồng', 'crack', 'wrede' ,'dbpedia','ontology', 'tmf', 'vhdl',
        'hdl', 'jsp', 'pđt', 'lisp', 'json', 'cpp', 'itp', 'forum', 'embeded', 'system', 'embeded system', 'embedded', 'titanium', 'blackbery', 'zun', 'phonegap', 'tizen', 'je', 'mediafire','toeic', 'ghz', 'cpu', 'module', 'datapath', 'papers', 'daa', 'dijktra', 'oracal', 'database', 'access', 'netbean', 'facebook', 'hackerrankdotcom', 'sort', 'multiagent', 'th', 'contemn', 'dbms', 'html', 'php', 'heapsort', 'khmtdotuitdotedudotvn', 'vdotv', 'engine', 'download','input', 'output', 'wtf','forum', 'poison', 'uit', 'career', 'version', 'outdoor', 'coursedotuitdotedudotvn', 'mini', 'matlab', 'standford', 'name', 'size', 'framework', 'ucla', 'comment', 'is','we','serverside', 'cassette', 'ios', 'android', 'scrum', 'itdote', 'xml', 'photo', 'down', 'unikey', '3dsmax', 'firmware', 'km','hackerrank', 'projectbase', 'er', 'gay', 'feed', 'mác – lênin', 'mác', 'lênin', 'coursedotuitdotedudotvn', 'nfc', 'chip', 'oi', 'ht', 'ubuntu', 'linux', 'it', 'wecode','oop', 'hướng đối tượng','macbook', 'a_z', 'môn_toán', 'môn_lý'
    ]
# Clean stopwords list
stopwords_small = [line.strip().replace(" ", "_") for line in stopwords_small]
stopwords_small.append('dot')
# stopwords_small.append('giảng_viên')
for item in list_rare_words_1:
    stopwords_small.append(item)

def preprocess_text_vietnamese_to_tokens_simple(text):
    """
    Preprocess Vietnamese text by normalizing accents, removing unwanted elements,
    and tokenizing.
    :param text: str, input Vietnamese text
    :return: str, processed text
    """
    # 0. Normalize Unicode accents (remove diacritics)
    text = covert_unicode(text)

    # 1. Normalize Vietnamese accents using existing functions
    text = chuan_hoa_dau_cau_tieng_viet(text)

    # 2. Chuyển đổi văn bản thành chữ thường
    text = text.lower()

    # 3. Thực hiện ánh xạ các cụm từ theo từ điển
    mapping_dict = {
        "thầy giáo": "giảng viên",
        "cô giáo": "giảng viên",
        "thầy": "giảng viên",
        "cô": "giảng viên",
        "giáo viên": "giảng viên",
        'vói': 'với',
        'giời': 'giờ',
        'nhiệt hình': 'nhiệt tình',
        'side': 'slide',
        'tân tình': 'tận tình',
        'teacher': 'giảng viên',
        'sadcolon': 'colonsad',
        'vi dụ': 'ví dụ',
        'easy': 'dễ',
        'so vời': 'so với',
        'tâp': 'tập',
        'av': 'tiếng anh',
        'nhannh': 'nhanh',
        'h': 'giờ',
        'đc': 'được',
        'dc': 'được',
        'smilesmile': 'smile',
        'tron': 'trong',
        'thướng': 'hướng',
        'nghung': 'nhưng',
        'chon': 'chọn',
        'them': 'thêm',
        'day': 'giảng dạy',
        'midterm': 'giữa kỳ',
        'vi': 'vì',
        'quýêt': 'quyết',
        'teamwork': 'làm việc nhóm',
        'over time': 'quá giờ',
        'overtime': 'quá giờ',
        'ot': 'quá giờ',
        'outcome': 'mục tiêu',
        'basic': 'cơ bản',
        'check': 'kiểm tra',
        'gmail': 'email',
        'mail': 'email',
        'teen': 'trẻ',
        'style': 'phong cách',
        'topic': 'chủ đề',
        'file': 'tài liệu',
        'slides': 'tài liệu',
        'slide': 'tài liệu',
        'giáo trình': 'tài liệu',
        'slile': 'tài liệu',
        'silde': 'tài liệu',
        'web':'website',
        'online': 'website',
        'nope': 'không',
        'class': 'lớp học',
        'fitted': 'phù hợp',
        'chair': 'ghế',
        'table': 'bàn',
        'grammar': 'ngữ pháp',
        'speaking': 'kỹ năng nói',
        'listening': 'kỹ năng nghe',
        'listenning': 'kỹ năng nghe',
        'reading': 'kỹ năng đọc',
        'good': 'tốt',
        'english': 'tiếng anh',
        'bad': 'tệ',
        'feedback': 'phản hồi',
        'elab': 'lab',
        'group':'nhóm',
        'mic': 'microphone',
        'debate': 'thảo luận',
        'test': 'kiểm tra',
        'thưc': 'thực',
        'courses': 'khóa học',
        'funny': 'vui vẻ',
        'internet': 'mạng',
        'perfect': 'hoàn hảo',
        'quoa': 'qua',
        'thanks': 'cảm ơn',
        'thankss': 'cảm ơn',
        'eemail': 'email',
        'very': 'rất',
        'dạy':'giảng dạy',
        'full': 'đầy đủ'
        # Thêm các cặp ánh xạ khác nếu cần
    }
    for original, replacement in mapping_dict.items():
        # Sử dụng regex để đảm bảo chỉ thay thế các cụm từ chính xác
        pattern = r'\b' + re.escape(original) + r'\b'
        text = re.sub(pattern, replacement, text)

    # 4. Loại bỏ dấu câu
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 5. Loại bỏ số
    text = re.sub(r'\d+', '', text)

    # 6. Loại bỏ khoảng trắng thừa ở đầu và cuối
    text = text.strip()

    # 7. Chuẩn hóa văn bản
    text = text_normalize(text)

    # 8. Loại bỏ biểu tượng cảm xúc (emojis)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002500-\U00002BEF"  # Chinese characters
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+",
        flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # 9. Loại bỏ các tiền tố cụ thể nhưng giữ lại hậu tố
    text = re.sub(r'\bcolon(\w+)\b', r'\1', text)  # Loại bỏ tiền tố "colon" nhưng giữ lại hậu tố
    text = re.sub(r'\bdoubledot(\w+)\b', r'\1', text)  # Loại bỏ tiền tố "doubledot" nhưng giữ lại hậu tố
    # 10. Định nghĩa các tiền tố cần loại bỏ
    prefixes_to_remove = ['wzjwz', 'wwzjwz']
    text = re.sub(r'\b\w*smile\w*\b', 'smile', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*surprise\w*\b', 'surprise', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*sad\w*\b', 'sad', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w*contemn\w*\b', 'contemn', text, flags=re.IGNORECASE)

    # 11. Loại bỏ các từ bắt đầu bằng bất kỳ tiền tố nào trong danh sách
    for prefix in prefixes_to_remove:
        text = re.sub(r'\b' + re.escape(prefix) + r'\w*\b', '', text)

    # 12. Loại bỏ các từ hiếm
    list_rare_words = [
        'fraction', 'altera', 'quad', 'cnpm', 'kaydotvn', 'daadotuitdotedudotvn', 'bohm', 'dotnet',
        'visual studio code', 'visual studio', 'visual code', 'visual', 'mutton quad', 'dreamweaver',
        'vertical fragmentation', 'i-ta-li-a', 'đbcl', 'crt', 'javabeans', 'if', 'for', 'while',
        'struct', 'pl', 'windows phone', 'pm', 'assembly', 'scrum_project', 'scrum project',
        'css', 'jquery', 'vector', 'gb', 'ram', 'ram', '...', 'codefun', 'egov', 'java', 'vmware',
        'placement', 'fork', 'round robin', 'patin', 'pattern', 'serverside', 'dotnet', 'c++',
        'javascript', 'ch', 'uit khoa', 'proteus', 'console', 'form', 'vật lý học', 'aep',
        'servlet', 'skype', 'doubledot', 'poster', 'everything', 'amp', 'app', 'seo', 'app',
        'progressive web', 'xim', 'win server', 'p', 's', 'p s', 'dropbox',
        'cplusplus', 'socket', 'sub', 'switch',
        'severside', 'network programing with csharp', 'dfd', 'naives bayes', 'naive', 'bayes', 'cs', 'js', 'max', 'elg', 'fix', 'proxy',
        'hub', 'bridge','switch', 'windows', 'turnitindotcom', 'extensive reading', 'reading', 'extensive', 'search', 'quick', 'file header','paper',
        'directx', 'windows', 'linux', 'vote', 'itdotf', 'router', 'silverlight', 'đa luồng', 'crack', 'wrede' ,'dbpedia','ontology', 'tmf', 'vhdl',
        'hdl', 'jsp', 'pđt', 'lisp', 'json', 'cpp', 'itp', 'forum', 'embeded', 'system', 'embeded system', 'embedded', 'titanium', 'blackbery', 'zun', 'phonegap', 'tizen', 'je', 'mediafire','toeic', 'ghz', 'cpu', 'module', 'datapath', 'papers', 'daa', 'dijktra', 'oracal', 'database', 'access', 'netbean', 'facebook', 'hackerrankdotcom', 'sort', 'multiagent', 'th', 'contemn', 'dbms', 'html', 'php', 'heapsort', 'khmtdotuitdotedudotvn', 'vdotv', 'engine', 'download','input', 'output', 'wtf','forum', 'poison', 'uit', 'career', 'version', 'outdoor', 'coursedotuitdotedudotvn', 'mini', 'matlab', 'standford', 'name', 'size', 'framework', 'ucla', 'comment', 'is','we','serverside', 'cassette', 'ios', 'android', 'scrum', 'itdote', 'xml', 'photo', 'down', 'unikey', '3dsmax', 'firmware', 'km','hackerrank', 'projectbase', 'er', 'gay', 'feed', 'mác – lênin', 'mác', 'lênin', 'coursedotuitdotedudotvn', 'nfc', 'chip', 'oi', 'ht', 'ubuntu', 'linux', 'it', 'wecode','oop', 'hướng đối tượng','macbook', 'a_z', 'vscode', 'môn_toán', 'môn_lý'
    ]

    # Remove rare words using regex
    rarewords_pattern = r'\b(?:' + '|'.join(re.escape(word) for word in list_rare_words) + r')\b'
    text = re.sub(rarewords_pattern, '', text)

    # 13. Token hóa văn bản
    tokens = word_tokenize(text, format='text').split()
    tokens = [token for token in tokens if token not in stopwords_small]

    list_ai_stopwords = [
        'và', 'cho', 'với', 'của', 'các', 'để', 'trong', 'về', 'thì', 'trên', 'vào',
        'một', 'ra', 'này', 'lại', 'mà', 'như', 'gì', 'cũng', 'lên', 'chỉ', 'đến',
        'khi', 'những', 'phòng', 'phần', 'buổi', 'đi', 'việc', 'là', 'có', 'nên',
        'môn', 'thêm', 'còn', 'nhưng', 'cách', 'giờ', 'thời_gian', 'nội_dung',
        'luôn', 'ít', 'vì', 'tiếng', 'anh', 'vấn_đề', 'phải', 'đó', 'có_thể', 'hay', 'được'
    ]

    tokens = [token for token in tokens if token not in list_ai_stopwords]

    # 14. Loại bỏ các token rỗng nếu có
    tokens = [token for token in tokens if token.strip()]

    # 15. Ghép các token lại thành chuỗi văn bản đã xử lý
    text = ' '.join(tokens)
    tokens = word_tokenize(text)
    return tokens


In [89]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

# Hàm giả định preprocess_text_vietnamese_to_tokens và preprocess_text_vietnamese_to_text đã được định nghĩa

# Xử lý DataFrame train
# df_train = pd.read_csv('UIT-VSFC_train_cleaned.csv')
df_train = pd.read_csv('UIT-VSFC_train.csv')
df_train = df_train.dropna(subset=['sents'])  # Loại bỏ các dòng có NaN trong 'sents'
df_train['tokens'] = df_train['sents'].apply(preprocess_text_vietnamese_to_tokens_simple)
df_train['sents'] = df_train['sents'].apply(preprocess_text_vietnamese_to_text_simple)
df_train = df_train[df_train['sents'].str.strip() != '']  # Loại bỏ các dòng có 'sents' rỗng
tokens_1 = list(df_train['tokens'])

# Xử lý DataFrame test
# df_test = pd.read_csv('UIT-VSFC_test_cleaned.csv')
df_test = pd.read_csv('UIT-VSFC_test.csv')
df_test = df_test.dropna(subset=['sents'])  # Loại bỏ các dòng có NaN trong 'sents'
df_test['tokens'] = df_test['sents'].apply(preprocess_text_vietnamese_to_tokens_simple)
df_test['sents'] = df_test['sents'].apply(preprocess_text_vietnamese_to_text_simple)
df_test = df_test[df_test['sents'].str.strip() != '']  # Loại bỏ các dòng có 'sents' rỗng
tokens_2 = list(df_test['tokens'])

# Xử lý DataFrame dev
# df_dev = pd.read_csv('UIT-VSFC_dev_cleaned.csv')
df_dev = pd.read_csv('UIT-VSFC_dev.csv')
df_dev = df_dev.dropna(subset=['sents'])  # Loại bỏ các dòng có NaN trong 'sents'
df_dev['tokens'] = df_dev['sents'].apply(preprocess_text_vietnamese_to_tokens_simple)
df_dev['sents'] = df_dev['sents'].apply(preprocess_text_vietnamese_to_text_simple)
df_dev = df_dev[df_dev['sents'].str.strip() != '']  # Loại bỏ các dòng có 'sents' rỗng
tokens_3 = list(df_dev['tokens'])

# Chuyển pandas DataFrame thành Dask DataFrame và thực hiện thay thế
def clean_dataframe(df):
    dask_df = dd.from_pandas(df, npartitions=4)
    dask_df['tokens'] = dask_df['tokens'].str.replace("_", " ", regex=False)
    dask_df['sents'] = dask_df['sents'].str.replace("_", " ", regex=False)
    return dask_df.compute()

# df_train = clean_dataframe(df_train)
# df_test = clean_dataframe(df_test)
# df_dev = clean_dataframe(df_dev)

# Lưu DataFrame đã làm sạch
df_train.to_csv('UIT-VSFC_train_cleaned_simple.csv', index=False)
df_test.to_csv('UIT-VSFC_test_cleaned_simple.csv', index=False)
df_dev.to_csv('UIT-VSFC_dev_cleaned_simple.csv', index=False)

In [90]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# Giả sử df là DataFrame chứa cột 'text' với các văn bản đã được tiền xử lý và token hóa
# Ví dụ:
# df = pd.DataFrame({'text': ['this is a sentence', 'another sentence', ...]})

# Sử dụng CountVectorizer để chuyển đổi văn bản thành ma trận đếm từ
count = CountVectorizer()
word_count = count.fit_transform(df_train['sents'])

# Tính toán TF-IDF sử dụng TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)

# Lấy các từ và trọng số IDF
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count.get_feature_names_out(), columns=["idf_weights"])
df_idf = df_idf.sort_values(by=['idf_weights'])

# Tính ma trận TF-IDF
tf_idf_vector = tfidf_transformer.transform(word_count)

# Lấy tên các tính năng (tokens)
feature_names = count.get_feature_names_out()

# Tính tần suất từ (số lần xuất hiện trong toàn bộ tài liệu)
word_counts = np.array(word_count.sum(axis=0)).flatten()
df_freq = pd.DataFrame({
    'Token': feature_names,
    'Frequency': word_counts
})

# Tính điểm TF-IDF trung bình cho mỗi từ
tfidf_scores = np.array(tf_idf_vector.mean(axis=0)).flatten()
df_tfidf = pd.DataFrame({
    'Token': feature_names,
    'TF-IDF': tfidf_scores
})

# Kết hợp tần suất và TF-IDF
df_combined = pd.merge(df_freq, df_tfidf, on='Token')

# Sắp xếp DataFrame theo tần suất giảm dần
df_combined = df_combined.sort_values(by='Frequency', ascending=False)

# Lưu DataFrame vào tệp CSV (tùy chọn)
df_combined.to_csv('token_frequencies_train_tfidf_simple.csv', index=False, encoding='utf-8-sig')

print("Đã tính toán tần suất và TF-IDF cho các từ.")


Đã tính toán tần suất và TF-IDF cho các từ.


In [91]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# Giả sử df là DataFrame chứa cột 'text' với các văn bản đã được tiền xử lý và token hóa
# Ví dụ:
# df = pd.DataFrame({'text': ['this is a sentence', 'another sentence', ...]})

# Sử dụng CountVectorizer để chuyển đổi văn bản thành ma trận đếm từ
count = CountVectorizer()
word_count = count.fit_transform(df_test['sents'])

# Tính toán TF-IDF sử dụng TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)

# Lấy các từ và trọng số IDF
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count.get_feature_names_out(), columns=["idf_weights"])
df_idf = df_idf.sort_values(by=['idf_weights'])

# Tính ma trận TF-IDF
tf_idf_vector = tfidf_transformer.transform(word_count)

# Lấy tên các tính năng (tokens)
feature_names = count.get_feature_names_out()

# Tính tần suất từ (số lần xuất hiện trong toàn bộ tài liệu)
word_counts = np.array(word_count.sum(axis=0)).flatten()
df_freq = pd.DataFrame({
    'Token': feature_names,
    'Frequency': word_counts
})

# Tính điểm TF-IDF trung bình cho mỗi từ
tfidf_scores = np.array(tf_idf_vector.mean(axis=0)).flatten()
df_tfidf = pd.DataFrame({
    'Token': feature_names,
    'TF-IDF': tfidf_scores
})

# Kết hợp tần suất và TF-IDF
df_combined = pd.merge(df_freq, df_tfidf, on='Token')

# Sắp xếp DataFrame theo tần suất giảm dần
df_combined = df_combined.sort_values(by='Frequency', ascending=False)

# Lưu DataFrame vào tệp CSV (tùy chọn)
df_combined.to_csv('token_frequencies_test_tfidf_simple.csv', index=False, encoding='utf-8-sig')

print("Đã tính toán tần suất và TF-IDF cho các từ.")


Đã tính toán tần suất và TF-IDF cho các từ.


In [92]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# Giả sử df là DataFrame chứa cột 'text' với các văn bản đã được tiền xử lý và token hóa
# Ví dụ:
# df = pd.DataFrame({'text': ['this is a sentence', 'another sentence', ...]})

# Sử dụng CountVectorizer để chuyển đổi văn bản thành ma trận đếm từ
count = CountVectorizer()
word_count = count.fit_transform(df_dev['sents'])

# Tính toán TF-IDF sử dụng TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)

# Lấy các từ và trọng số IDF
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count.get_feature_names_out(), columns=["idf_weights"])
df_idf = df_idf.sort_values(by=['idf_weights'])

# Tính ma trận TF-IDF
tf_idf_vector = tfidf_transformer.transform(word_count)

# Lấy tên các tính năng (tokens)
feature_names = count.get_feature_names_out()

# Tính tần suất từ (số lần xuất hiện trong toàn bộ tài liệu)
word_counts = np.array(word_count.sum(axis=0)).flatten()
df_freq = pd.DataFrame({
    'Token': feature_names,
    'Frequency': word_counts
})

# Tính điểm TF-IDF trung bình cho mỗi từ
tfidf_scores = np.array(tf_idf_vector.mean(axis=0)).flatten()
df_tfidf = pd.DataFrame({
    'Token': feature_names,
    'TF-IDF': tfidf_scores
})

# Kết hợp tần suất và TF-IDF
df_combined = pd.merge(df_freq, df_tfidf, on='Token')

# Sắp xếp DataFrame theo tần suất giảm dần
df_combined = df_combined.sort_values(by='Frequency', ascending=False)

# Lưu DataFrame vào tệp CSV (tùy chọn)
df_combined.to_csv('token_frequencies_dev_tfidf_simple.csv', index=False, encoding='utf-8-sig')

print("Đã tính toán tần suất và TF-IDF cho các từ.")


Đã tính toán tần suất và TF-IDF cho các từ.


In [93]:
def get_stopwords(documents, threshold=0.1):
    """
    :param documents: list of documents
    :param threshold:
    :return: list of words has idf <= threshold
    """
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(documents)
    features = tfidf.get_feature_names_out()
    stopwords = []
    print(min(tfidf.idf_), max(tfidf.idf_), len(features))
    for index, feature in enumerate(features):
        if tfidf.idf_[index] <= threshold:
            stopwords.append(feature)
    return stopwords

test_stopwords = get_stopwords(df_train['sents'])
print(test_stopwords)

1.697886110007817 9.650149418664864 3736
[]


In [94]:
# import os
# import re
# import string
# import pandas as pd

# # underthesea
# from underthesea import word_tokenize
# from underthesea import text_normalize

# # joblib để chạy đa tiến trình
# from joblib import Parallel, delayed

# # gensim
# from gensim.models import Word2Vec
# from gensim.models.callbacks import CallbackAny2Vec
# from tqdm import tqdm
# tqdm.pandas(desc="progress-bar")
# import gensim
# from gensim.models.word2vec import Word2Vec
# from gensim.models.doc2vec import TaggedDocument
# import multiprocessing
# from sklearn import utils
# from gensim.models import KeyedVectors


# cores = multiprocessing.cpu_count()
# model_ug_cbow = Word2Vec(
#         sg=0,
#         vector_size=300,
#         window=7,
#         min_count=10,
#         workers=cores,
# )
# model_ug_cbow.build_vocab(df_train['tokens'])
# model_ug_cbow.train(df_train['tokens'], total_examples=len(df_train['tokens']), epochs=30)

# model_ug_sg = Word2Vec(
#         sg=1,
#         vector_size=300,
#         window=7,
#         min_count=10,
#         workers=cores,
#     )
# model_ug_sg.build_vocab(df_train['tokens'])
# model_ug_sg.train(df_train['tokens'], total_examples=len(df_train['tokens']), epochs=30)

# model_ug_cbow.save('model_ug_cbow_simple.word2vec')
# model_ug_sg.save('model_ug_sg_simple.word2vec')

# model_ug_cbow.wv.save_word2vec_format('model_ug_cbow_simple.bin', binary=True)
# model_ug_sg.wv.save_word2vec_format('model_ug_sg_simple.bin', binary=True)

# print(len(df_train['tokens']))

In [95]:
from gensim.models.doc2vec import TaggedDocument


def labelize_text_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t, [prefix + '_%s' % i]))
    return result

In [96]:
tmp = df_train['tokens']

In [97]:
all_x = pd.concat([tmp])
all_x_w2v = labelize_text_ug(all_x, 'all')
print(all_x_w2v)

[TaggedDocument(words=['tài_liệu', 'tài_liệu', 'đầy_đủ'], tags=['all_0']), TaggedDocument(words=['nhiệt_tình', 'giảng', 'giảng_dạy', 'gần_gũi', 'sinh_viên'], tags=['all_1']), TaggedDocument(words=['học', 'đầy_đủ', 'đầy_đủ điểm', 'chuyên', 'cần'], tags=['all_2']), TaggedDocument(words=['chưa', 'áp_dụng', 'công_nghệ_thông_tin', 'thiết_bị', 'hỗ_trợ', 'giảng', 'giảng_dạy'], tags=['all_3']), TaggedDocument(words=['giảng_viên', 'giảng_bài', 'nhiều', 'bài_tập', 'ví_dụ', 'ngay', 'lớp'], tags=['all_4']), TaggedDocument(words=['giảng_viên', 'đảm_bảo', 'lớp', 'tích_cực', 'trả_lời', 'câu', 'hỏi', 'sinh_viên', 'thường_xuyên', 'đặt', 'câu', 'hỏi', 'sinh_viên'], tags=['all_5']), TaggedDocument(words=['em', 'sẽ', 'nợ', 'em', 'sẽ', 'học', 'ở', 'học_kỳ', 'kế_tiếp'], tags=['all_6']), TaggedDocument(words=['thời_lượng', 'học', 'quá', 'dài', 'không', 'đảm_bảo', 'tiếp_thu', 'hiệu_quả'], tags=['all_7']), TaggedDocument(words=['môn_học', 'thiếu', 'trọng_tâm', 'hầu_như', 'chung_chung', 'khái_quát', 'khiến', 's

In [98]:
from sklearn import utils
from tqdm import tqdm
from gensim.models import Word2Vec
import multiprocessing

cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, vector_size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha


100%|██████████| 11421/11421 [00:00<00:00, 2342452.13it/s]


In [99]:
model_ug_sg = Word2Vec(sg=1, vector_size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])
for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha

100%|██████████| 11421/11421 [00:00<00:00, 1419983.58it/s]


In [117]:
from sklearn import utils
from tqdm import tqdm
from gensim.models import FastText
import multiprocessing

cores = multiprocessing.cpu_count()

# Initialize FastText model
model_ug_cbow_fasttext = FastText(
    sg=0,  # CBOW model
    vector_size=300,  # Size of word vectors
    window=15,  # Context window size
    min_count=10,  # Minimum count of words
    workers=cores,  # Number of CPU cores
    alpha=0.065,  # Initial learning rate
    min_alpha=0.065  # Minimum learning rate
)

# Build vocabulary
model_ug_cbow_fasttext.build_vocab([x.words for x in tqdm(all_x_w2v)])

# Train model
for epoch in range(30):
    model_ug_cbow_fasttext.train(
        utils.shuffle([x.words for x in tqdm(all_x_w2v)]),
        total_examples=len(all_x_w2v),
        epochs=1
    )
    model_ug_cbow_fasttext.alpha -= 0.002  # Decrease learning rate
    model_ug_cbow_fasttext.min_alpha = model_ug_cbow_fasttext.alpha  # Update minimum learning rate


100%|██████████| 11421/11421 [00:00<00:00, 2243812.17it/s]


In [124]:
from sklearn import utils
from tqdm import tqdm
from gensim.models import FastText
import multiprocessing

cores = multiprocessing.cpu_count()

# Initialize FastText model
model_ug_sg_fasttext = FastText(
    sg=1,  # CBOW model
    vector_size=300,  # Size of word vectors
    window=15,  # Context window size
    min_count=1,  # Minimum count of words
    workers=cores,  # Number of CPU cores
    alpha=0.065,  # Initial learning rate
    min_alpha=0.065  # Minimum learning rate
)

# Build vocabulary
model_ug_sg_fasttext.build_vocab([x.words for x in tqdm(all_x_w2v)])

# Train model
for epoch in range(30):
    model_ug_sg_fasttext.train(
        utils.shuffle([x.words for x in tqdm(all_x_w2v)]),
        total_examples=len(all_x_w2v),
        epochs=1
    )
    model_ug_sg_fasttext.alpha -= 0.002  # Decrease learning rate
    model_ug_sg_fasttext.min_alpha = model_ug_sg_fasttext.alpha  # Update minimum learning rate


100%|██████████| 11421/11421 [00:00<00:00, 1995881.25it/s]


In [129]:
from gensim.models import FastText
from sklearn import utils

import multiprocessing
import numpy as np

cores = multiprocessing.cpu_count()

# --- Cải tiến chính ---
# 1. Sử dụng Skip-gram (sg=1) cho dữ liệu ít
# 2. Điều chỉnh learning rate và lịch trình huấn luyện
# 3. Thêm xử lý từ ghép và các tối ưu cho tiếng Việt



# Khởi tạo model
model_ug_sg_fasttext = FastText(
    sg=1,              # Skip-gram phù hợp hơn với dataset nhỏ
    vector_size=150,   # Giảm kích thước vector cho dữ liệu ít
    window=8,          # Cửa sổ ngữ cảnh vừa phải cho câu ngắn
    min_count=5,       # Bỏ qua các từ xuất hiện dưới 5 lần
    workers=cores-1,   # Để lại 1 core cho các task khác
    alpha=0.025,       # Learning rate ban đầu
    min_alpha=0.001,   # Learning rate tối thiểu
    sample=1e-5,       # Ngưỡng downsampling
    negative=15,       # Số lượng negative samples
    ns_exponent=0.5,   # Điều chỉnh phân phối negative sampling
    hashfxn=np.random.default_rng().bit_generator.randint # Fix seed reproducibility
)

# Xây dựng vocab
model_ug_sg_fasttext.build_vocab(
    corpus_iterable=all_x_w2v,
    progress_per=10000
)

# Huấn luyện với lịch trình learning rate
for epoch in range(30):
    # Xáo trộn dữ liệu mỗi epoch
    shuffled_data = utils.shuffle(all_x_w2v)

    # Huấn luyện với thanh progress bar
    with tqdm(total=len(shuffled_data), desc=f'Epoch {epoch+1}') as pbar:
        model_ug_sg_fasttext.train(
            corpus_iterable=shuffled_data,
            total_examples=len(shuffled_data),
            epochs=1,
            start_alpha=model_ug_sg_fasttext.alpha,
            end_alpha=model_ug_sg_fasttext.min_alpha,
            compute_loss=True,
            queue_factor=2,  # Tăng kích thước batch
            report_delay=60
        )
        pbar.update(len(shuffled_data))

    # In loss sau mỗi epoch
    print(f'Epoch {epoch+1} - Loss: {model_ug_sg_fasttext.get_latest_training_loss():.2f}')

    # Điều chỉnh learning rate theo cosine schedule
    model_ug_sg_fasttext.alpha = model_ug_sg_fasttext.min_alpha + (
        (0.025 - model_ug_sg_fasttext.min_alpha) *
        (1 + np.cos(np.pi * epoch / 30)) / 2
    )

# Lưu model
model_ug_sg_fasttext.save("fasttext_sentiment.model")

# --- Kiểm tra kết quả ---
# Test với các từ quan trọng
test_words = ['nhiệt_tình', 'khó', 'dễ_hiểu', 'bài_tập']
for word in test_words:
    if word in model_ug_sg_fasttext.wv:
        print(f"Top 10 từ tương đồng với '{word}':")
        print([w[0] for w in model_ug_sg_fasttext.wv.most_similar(word, topn=10)])
    else:
        print(f"Từ '{word}' không có trong vocab")

AttributeError: 'numpy.random._pcg64.PCG64' object has no attribute 'randint'

In [125]:
import numpy as np

embeddings_index = {}

for w in model_ug_sg_fasttext.wv.key_to_index.keys():

    # embeddings_index[w] = np.append(word2vec_cbow.wv[w], word2vec_sg.wv[w])
    embeddings_index[w] = model_ug_sg_fasttext.wv[w]


print('Found %s word vectors.' % len(embeddings_index))

Found 4375 word vectors.


In [126]:
words = model_ug_sg_fasttext.wv.key_to_index.keys()
print(words)
vocab_size = len(words)
print("Vocab size", vocab_size)

dict_keys(['giảng_viên', 'giảng_dạy', 'sinh_viên', 'không', 'rất', 'nhiệt_tình', 'hiểu', 'nhiều', 'giảng', 'dễ', 'học', 'bài_tập', 'bài', 'hơn', 'em', 'kiến_thức', 'thực_hành', 'cần', 'lớp', 'tốt', 'làm', 'môn_học', 'tài_liệu', 'quá', 'chưa', 'khó', 'tận_tâm', 'truyền_đạt', 'nói', 'tận_tình', 'lý_thuyết', 'hướng_dẫn', 'hơi', 'khá', 'học_sinh', 'giảng giảng_dạy', 'đúng', 'hiệu_quả', 'thực_tế', 'vui_vẻ', 'giúp', 'nhanh', 'đầy_đủ', 'vui_tính', 'ví_dụ', 'đồ_án', 'tạo', 'mới', 'cao', 'nghe', 'tiếp_thu', 'điểm', 'cung_cấp', 'phương_pháp', 'học_tập', 'theo', 'đã', 'trước', 'sự', 'thi', 'biết', 'nghỉ', 'rõ', 'quan_tâm', 'sử_dụng', 'khả_năng', 'đưa', 'ở', 'thắc_mắc', 'kỹ', 'sẽ', 'nhóm', 'khác', 'từ', 'hỏi', 'chi_tiết', 'mong', 'đảm_bảo', 'bạn', 'thân_thiện', 'gây', 'thấy', 'cụ_thể', 'giải_đáp', 'thường_xuyên', 'tự', 'sau', 'giảng_bài', 'người', 'thực_tiễn', 'nhà', 'kỹ_năng', 'máy', 'chất_lượng', 'rồi', 'giúp_đỡ', 'nào', 'tính', 'cuối', 'tới', 'bài_học', 'bị', 'rõ_ràng', 'vui', 'câu', 'chương_tr

In [102]:
model_ug_cbow.save('w2v_model_ug_cbow_100_simple_tokens.word2vec')
model_ug_sg.save('w2v_model_ug_sg_100_simple_tokens.word2vec')
model_ug_sg_fasttext.save('w2v_model_ug_sg_300_simple_tokens.bin')
model_ug_cbow_fasttext.save('w2v_model_ug_cbow_300_simple_tokens.bin')

In [103]:
print(df_train)

                                                   sents  sentiments  \
0                               tài_liệu tài_liệu đầy_đủ         2.0   
1           nhiệt_tình giảng giảng_dạy gần_gũi sinh_viên         2.0   
2                             học đầy_đủ điểm chuyên cần         0.0   
3      chưa áp_dụng công_nghệ_thông_tin thiết_bị hỗ_t...         0.0   
4      giảng_viên giảng bài nhiều bài_tập ví_dụ ngay lớp         2.0   
...                                                  ...         ...   
11421  game em học hai lần không qua quả_thật em rất ...         0.0   
11422                         em cảm_ơn giảng_viên nhiều         2.0   
11423                             giao bài_tập quá nhiều         0.0   
11424            giảng_viên giảng_dạy dễ hiểu nhiệt_tình         2.0   
11425  gói gọn tận_tình phù_hợp mọi trình_độ nhu_cầu ...         2.0   

                                                  tokens  
0                           [tài_liệu, tài_liệu, đầy_đủ]  
1      [nhiệt_tìn

In [104]:
augmented_data = []
for index, row in df_train.iterrows():
    try:
        # Directly use the tokens if they are already lists, otherwise split the string
        tokens = row['tokens']
        if isinstance(tokens, str):
            # Convert from string representation to list by removing brackets and splitting
            tokens = tokens.strip("[]").replace("'", "").split(", ")

        original_sentence = row['sents']
        sentiment = row['sentiments']

        # Generate sentences with synonyms
        for i, token in enumerate(tokens):
            synonyms = get_synonyms(token) or []
            for synonym in synonyms:
                # Create a new token list with the synonym replaced
                new_tokens = tokens[:]

                new_tokens[i] = synonym

                # Recreate the sentence
                new_sentence = ' '.join(new_tokens)

                # Append the new row
                augmented_data.append({
                    'sents': new_sentence,
                    'sentiments': sentiment,
                    'tokens': new_tokens
                })
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Convert augmented data into a DataFrame
augmented_df = pd.DataFrame(augmented_data)
print(augmented_df)

import dask.dataframe as dd

# Convert pandas DataFrame to Dask DataFrame
dask_df = dd.from_pandas(augmented_df, npartitions=4)
# Perform replacement
dask_df['sents'] = dask_df['sents'].str.replace("_", " ", regex=False)
# Convert back to pandas DataFrame and save
augmented_df = dask_df.compute()
augmented_df.to_csv("UIT-VSFC_train_tao_sinh_simple.csv", index=False)

                                                     sents  sentiments  \
0                                    hồ_sơ tài_liệu đầy_đủ         2.0   
1                                 văn_kiện tài_liệu đầy_đủ         2.0   
2                                  dữ_liệu tài_liệu đầy_đủ         2.0   
3                              tập_dữ_liệu tài_liệu đầy_đủ         2.0   
4                                  tư_liệu tài_liệu đầy_đủ         2.0   
...                                                    ...         ...   
1252216   gói gọn tận_tình phù_hợp mọi bậc nhu_cầu môn_học         2.0   
1252217   gói gọn tận_tình phù_hợp mọi mức nhu_cầu môn_học         2.0   
1252218   gói gọn tận_tình phù_hợp mọi cấp nhu_cầu môn_học         2.0   
1252219  gói gọn tận_tình phù_hợp mọi có_năng_lực nhu_c...         2.0   
1252220  gói gọn tận_tình phù_hợp mọi trình_độ cầu môn_học         2.0   

                                                    tokens  
0                                [hồ_sơ, tài_liệu,

In [105]:
augmented_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252221 entries, 0 to 1252220
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   sents       1252221 non-null  string 
 1   sentiments  1252221 non-null  float64
 2   tokens      1252221 non-null  string 
dtypes: float64(1), string(2)
memory usage: 303.5 MB


In [106]:
data_tao_sinh = pd.read_csv('UIT-VSFC_train_tao_sinh_simple.csv')

# Check the number of labels in each sentiment
label_counts = data_tao_sinh['sentiments'].value_counts()

# Get the number of neutral labels
neutral_count = label_counts[1]  # Neutral sentiment (1)

# Filter all neutral labels
neutral_data = data_tao_sinh[data_tao_sinh['sentiments'] == 1]

# Filter positive and negative labels and sample equal to the count of neutral
positive_data = data_tao_sinh[data_tao_sinh['sentiments'] == 2].sample(neutral_count, random_state=42)
negative_data = data_tao_sinh[data_tao_sinh['sentiments'] == 0].sample(neutral_count, random_state=42)

# Combine the sampled datasets
balanced_data = pd.concat([neutral_data, positive_data, negative_data])

# Shuffle the combined dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

balanced_data.to_csv('UIT-VSFC_train_tao_sinh_balanced_simple.csv', index=False)

In [107]:
balanced_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100533 entries, 0 to 100532
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   sents       100533 non-null  object 
 1   sentiments  100533 non-null  float64
 2   tokens      100533 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.3+ MB


In [108]:
data = pd.read_csv('UIT-VSFC_train_tao_sinh_simple.csv')

# Check the number of labels in each sentiment
label_counts = data['sentiments'].value_counts()

# Get the number of neutral labels
neutral_count = label_counts[1]  # Neutral sentiment (1)

# Filter all neutral labels
neutral_data = data[data_tao_sinh['sentiments'] == 1]

# Filter positive and negative labels and sample equal to the count of neutral
positive_data = data[data['sentiments'] == 2].sample(neutral_count, random_state=42)
negative_data = data[data['sentiments'] == 0].sample(neutral_count, random_state=42)

# Combine the sampled datasets
balanced_data = pd.concat([neutral_data, positive_data, negative_data])

# Shuffle the combined dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

balanced_data.to_csv('UIT-VSFC_train_balanced_simple.csv', index=False)

In [109]:
import pandas as pd

# Đọc dữ liệu
data = pd.read_csv('UIT-VSFC_train_cleaned_simple.csv')

# Tách dữ liệu theo từng nhãn
negative_data = data[data['sentiments'] == 0]
neutral_data  = data[data['sentiments'] == 1]
positive_data = data[data['sentiments'] == 2]

n_neg = len(negative_data)
n_neu = len(neutral_data)
n_pos = len(positive_data)

print("Số lượng mẫu (Negative, Neutral, Positive):", (n_neg, n_neu, n_pos))


Số lượng mẫu (Negative, Neutral, Positive): (5324, 456, 5641)


In [110]:
import numpy as np

def split_neg_pos(negative_df, positive_df, total_needed, random_state=42):
    """
    Lấy ra 1 lượng mẫu tổng cộng = total_needed từ negative_df và positive_df,
    giữ nguyên tỉ lệ negative:positive như trong dữ liệu gốc.
    """
    n_neg = len(negative_df)
    n_pos = len(positive_df)

    total_negpos = n_neg + n_pos
    # Tính tỉ trọng
    neg_ratio = n_neg / total_negpos
    pos_ratio = n_pos / total_negpos

    # Tính số lượng sẽ sample
    neg_needed = int(total_needed * neg_ratio)
    pos_needed = int(total_needed * pos_ratio)

    # Sample
    negative_sampled = negative_df.sample(neg_needed, random_state=random_state)
    positive_sampled = positive_df.sample(pos_needed, random_state=random_state)

    return pd.concat([negative_sampled, positive_sampled], ignore_index=True)

def shuffle_and_save(final_df, filename, random_state=42):
    """
    Shuffle dữ liệu và lưu ra file CSV.
    """
    final_df = final_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    final_df.to_csv(filename, index=False)
    print(f"Đã lưu: {filename} - kích thước {len(final_df)}")


In [111]:
import math

ratios = [0.1, 0.2, 0.3]

negpos_count = len(negative_data) + len(positive_data)

for r in ratios:
    desired_neu = int(r * negpos_count)  # Số neutral mong muốn so với full neg+pos

    if len(neutral_data) >= desired_neu:
        # Trường hợp neutral đang lớn hơn mức cần.
        # => Under-sample neutral để còn đúng desired_neu.
        final_neutral = neutral_data.sample(desired_neu, random_state=42)

        # Giữ nguyên toàn bộ negative và positive
        final_negpos = pd.concat([negative_data, positive_data], ignore_index=True)

    else:
        # Trường hợp neutral đang ít hơn mức cần => neutral không đủ
        # => Giữ nguyên neutral, under-sample (cắt bớt) neg+pos để neutral đạt tỉ lệ r.
        final_neutral = neutral_data  # Giữ hết neutral

        # Muốn final_neutral / final_negpos = r
        # => final_negpos = len(final_neutral) / r
        final_negpos_needed = int(len(final_neutral) / r)

        # Under-sample negative+positive xuống final_negpos_needed
        final_negpos = split_neg_pos(negative_data, positive_data, final_negpos_needed, random_state=42)

    # Kết hợp
    final_df = pd.concat([final_neutral, final_negpos], ignore_index=True)

    # Shuffle & Lưu
    fname = f"UIT-VSFC_train_undersampling_{int(r*100)}_simple.csv"
    shuffle_and_save(final_df, fname)


Đã lưu: UIT-VSFC_train_undersampling_10_simple.csv - kích thước 5015
Đã lưu: UIT-VSFC_train_undersampling_20_simple.csv - kích thước 2735
Đã lưu: UIT-VSFC_train_undersampling_30_simple.csv - kích thước 1975


In [112]:
negpos_count = len(negative_data) + len(positive_data)

for r in ratios:
    desired_neu = int(r * negpos_count)

    # Ở ví dụ này, ta giả sử muốn giữ nguyên toàn bộ negative + positive
    # rồi điều chỉnh neutral để đạt tỉ lệ.
    final_negpos = pd.concat([negative_data, positive_data], ignore_index=True)

    if len(neutral_data) == desired_neu:
        # Đúng bằng, giữ nguyên
        final_neutral = neutral_data

    elif len(neutral_data) < desired_neu:
        # neutral ít hơn mức cần => Oversample (sample with replacement)
        final_neutral = neutral_data.sample(desired_neu, replace=True, random_state=42)

    else:
        # neutral nhiều hơn mức cần => phải undersample neutral
        final_neutral = neutral_data.sample(desired_neu, random_state=42)

    # Kết hợp
    final_df = pd.concat([final_negpos, final_neutral], ignore_index=True)

    # Shuffle & Lưu
    fname = f"UIT-VSFC_train_oversampling_{int(r*100)}_simple.csv"
    shuffle_and_save(final_df, fname)


Đã lưu: UIT-VSFC_train_oversampling_10_simple.csv - kích thước 12061
Đã lưu: UIT-VSFC_train_oversampling_20_simple.csv - kích thước 13158
Đã lưu: UIT-VSFC_train_oversampling_30_simple.csv - kích thước 14254
